In [0]:
import os
import tensorflow as tf
import numpy as np
import pickle as pk

In [4]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/My Drive/neu_nets_style_project')
os.getcwd()

Mounted at /content/drive


'/content/drive/My Drive/neu_nets_style_project'

Mounted at /content/drive


'/content/drive/My Drive/neu_nets_style_project'

In [0]:
# Resets the computational graph
def reset_graph():
    if 'sess' in globals() and sess:
        sess.close()
    tf.reset_default_graph()

In [0]:

def Neural_network_model(
    n_nodes_hl1=500,
    n_nodes_hl2=500,
    n_nodes_hl3=500,
    keep_rate = 1,
    n_classes=7
    ):
  
    reset_graph()
    
    # Declarando las entradas y salidas
    x=tf.placeholder('float',[None,100,100,3])
    y=tf.placeholder('float',[None,7])
    
    #Imagenes 
    img = x#tf.reshape(x, shape=[-1, 100, 100, 3])
    # Declarando las variables 
    weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,3,4])),
                'W_conv2':tf.Variable(tf.random_normal([5,5,4,8])),
               'W_conv3':tf.Variable(tf.random_normal([5,5,8,16])),
               'W_conv4':tf.Variable(tf.random_normal([5,5,16,32])),
               'W_fc':tf.Variable(tf.random_normal([7*7*32,1024])),
               'W_fc2':tf.Variable(tf.random_normal([1024,512])),
               'W_fc3':tf.Variable(tf.random_normal([512,128])),
               'out':tf.Variable(tf.random_normal([128, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([4])),
                'b_conv2':tf.Variable(tf.random_normal([8])),
               'b_conv3':tf.Variable(tf.random_normal([16])),
              'b_conv4':tf.Variable(tf.random_normal([32])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
              'b_fc2':tf.Variable(tf.random_normal([512])),
              'b_fc3':tf.Variable(tf.random_normal([128])),
               'out':tf.Variable(tf.random_normal([n_classes]))}
   
    
    
    # Declarando la arquitectura
    l1 = tf.nn.conv2d(img, weights['W_conv1'], strides=[1,1,1,1], padding='SAME')
    l1 = tf.add(l1, biases['b_conv1'])
    l1 = tf.nn.relu(l1)
    l1 = tf.nn.max_pool(l1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    
    l2 = tf.nn.conv2d(l1, weights['W_conv2'], strides=[1,1,1,1], padding='SAME')
    l2 = tf.add(l2, biases['b_conv2'])
    l2 = tf.nn.relu(l2)
    l2 = tf.nn.max_pool(l2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    l3 = tf.nn.conv2d(l2, weights['W_conv3'], strides=[1,1,1,1], padding='SAME')
    l3 = tf.add(l3, biases['b_conv3'])
    l3 = tf.nn.relu(l3)
    l3 = tf.nn.max_pool(l3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    l4 = tf.nn.conv2d(l3, weights['W_conv4'], strides=[1,1,1,1], padding='SAME')
    l4 = tf.add(l4, biases['b_conv4'])
    l4 = tf.nn.relu(l4)
    l4 = tf.nn.max_pool(l4, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
    
    print(l4.shape)
    fc = tf.reshape(l4,[-1, 7*7*32])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)
    
    #fc2 = tf.reshape(fc,[-1, 1024])
    fc2 = tf.nn.relu(tf.matmul(fc, weights['W_fc2'])+biases['b_fc2'])
    fc2 = tf.nn.dropout(fc2, keep_rate)
    
    fc3 = tf.nn.relu(tf.matmul(fc2, weights['W_fc3'])+biases['b_fc3'])

    output = tf.matmul(fc3, weights['out'])+biases['out']

    # Declarando la funcion de costo y entrenamiento
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits_v2(logits=output
                                                                   , labels=y) )
    tf.summary.scalar("costo",cost)
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    summaries = tf.summary.merge_all()
    
    return dict(
              x=x,
              y=y,
              output=output,
              cost=cost,
              optimizer=optimizer,
              summaries=summaries,
              saver = tf.train.Saver()
              )

In [0]:
# Globlal Variables
random_seed = 762
np.random.seed(random_seed)

In [0]:
def train_neural_network(DNN, hm_epochs=10,train=[],test=[],save=False):

    with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
        sess.run(tf.global_variables_initializer())
        writer = tf.summary.FileWriter("./LOG")
        tf.summary.FileWriter.add_graph(writer,sess.graph)

        for epoch in range(hm_epochs):
            epoch_loss = 0
            accuracy = 0
            count = 0
            #print(os.getcwd)
            for filename in train:
                file = open( filename, "rb" )
                batch = pk.Unpickler(file).load()
                #print(type(batch))
                #print(len(batch))
                #print(batch[0][1:3])
                batch = np.array(batch)
                #print(np.shape(batch))
                #print(batch.shape)
                x = np.array([x for x in batch[:,0]])
                y = np.array([x for x in batch[:,1]])
                epoch_x, epoch_y = (x,y)
                feed_dict={DNN["x"]: epoch_x, 
                           DNN["y"]: epoch_y}
                _, c, prediction,y,summ   = sess.run([DNN["optimizer"], DNN["cost"]
                                                 , DNN["output"], DNN["y"],DNN["summaries"]], 
                                                feed_dict=feed_dict)
                epoch_loss += c
                correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
                count += 1

                accuracy += tf.reduce_mean(tf.cast(correct, 'float'))
            if (epoch%2 == 0):
                writer.add_summary(summ,epoch)
                
            if isinstance(save, str):
                ENCname="./SVE/"+save+".ckpt"
                DNN['saver'].save(sess, ENCname)
               
            accuracy = accuracy/count
            print('Epoch', epoch, 'completed out of',hm_epochs,'loss:',epoch_loss,"accuracy:",accuracy.eval())

            accuracy = 0
            count = 0
            for filename in test:
                file = open( filename, "rb" )
                batch = pk.Unpickler(file).load()
                batch = np.array(batch)
                x = np.array([x for x in batch[:,0]])
                y = np.array([x for x in batch[:,1]])
                test_x, test_y = x,y

                prediction,y   = sess.run([DNN["output"], DNN["y"]], feed_dict={DNN["x"]:test_x, DNN["y"]:test_y})
                correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
                accuracy += tf.reduce_mean(tf.cast(correct, 'float'))
                count += 1
            accuracy = accuracy / count
            print('Accuracy:',accuracy.eval())


In [0]:
import re
file_list_train = [ os.path.join('good_data',str_file) for str_file in os.listdir('good_data') if re.search(r'train', str_file) ]
file_list_test = [ os.path.join('good_data',str_file) for str_file in os.listdir('good_data') if re.search(r'test', str_file) ]

In [0]:
train = file_list_train
test = file_list_test

In [11]:
train

['good_data/train_64_80.pyData',
 'good_data/train_48_64.pyData',
 'good_data/train_32_48.pyData',
 'good_data/train_16_32.pyData',
 'good_data/train_0_16.pyData',
 'good_data/train_80_96.pyData',
 'good_data/train_368_384.pyData',
 'good_data/train_352_368.pyData',
 'good_data/train_336_352.pyData',
 'good_data/train_320_336.pyData',
 'good_data/train_304_320.pyData',
 'good_data/train_288_304.pyData',
 'good_data/train_272_288.pyData',
 'good_data/train_256_272.pyData',
 'good_data/train_240_256.pyData',
 'good_data/train_224_240.pyData',
 'good_data/train_208_224.pyData',
 'good_data/train_192_208.pyData',
 'good_data/train_176_192.pyData',
 'good_data/train_160_176.pyData',
 'good_data/train_144_160.pyData',
 'good_data/train_128_144.pyData',
 'good_data/train_112_128.pyData',
 'good_data/train_96_112.pyData',
 'good_data/train_688_704.pyData',
 'good_data/train_672_688.pyData',
 'good_data/train_656_672.pyData',
 'good_data/train_640_656.pyData',
 'good_data/train_624_640.pyData',

In [12]:
test

['good_data/test_0_346.pyData']

In [13]:
DNN=Neural_network_model()
train_neural_network(DNN, 10, train, test, 'log_tf_board')

Instructions for updating:
Colocations handled automatically by placer.
(?, 7, 7, 32)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Epoch 0 completed out of 10 loss: 7670467262.0 accuracy: 0.21976997
Accuracy: 0.20119761
Epoch 1 completed out of 10 loss: 2867787504.0 accuracy: 0.27745146
Accuracy: 0.23353294
Epoch 2 completed out of 10 loss: 1845572689.625 accuracy: 0.3086985
Accuracy: 0.2251497
Epoch 3 completed out of 10 loss: 1331306360.0 accuracy: 0.33904174
Accuracy: 0.23952095
Epoch 4 completed out of 10 loss: 1008828530.75 accuracy: 0.3907765
Accuracy: 0.2251497
Epoch 5 completed out of 10 loss: 759262130.875 accuracy: 0.44134468
Accuracy: 0.22035928
Epoch 6 completed out of 10 loss: 626875459.078125 accuracy: 0.47646686
Accuracy: 0.1988024
Epoch 7 completed out of 10 loss: 469772225.71875 accuracy: 0.534698
Accuracy: 0.20838323
Epoch 8 completed out of 10 loss: 463217995.203125 accuracy: 0.5297871
Accuracy: 0.